### Objectives
You will be able to:

- Describe the relationship between aggregate functions and GROUP BY statements
- Use GROUP BY statements in SQL to apply aggregate functions like: COUNT, MAX, MIN, and SUM
- Create an alias in a SQL query
- Use the HAVING clause to compare different aggregates
- Compare the difference between the WHERE and HAVING clause

### Connecting to the Database
As usual, start by creating a connection to the database. We will also import pandas in order to display the results in a convenient format.

In [1]:
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.connect('data/data.sqlite')

### GROUP BY and Aggregate Functions
Let's start by looking at some GROUP BY statements to aggregate our data. The GROUP BY clause groups records into summary rows and returns one record for each group.

Typically, GROUP BY also involves an aggregate function (COUNT, AVG, etc.).

Lastly, GROUP BY can group by one column or multiple columns.

### Count of Customers by Country
One of the most common uses of GROUP BY is to count the number of records in each group. To do that, we'll also use the COUNT aggregate function.

In [3]:
q = '''
SELECT country,COUNT(*) from customers
group by country
'''
# Displaying just the first 10 countries for readability
pd.read_sql(q,conn).head(10)

,country,COUNT(*)
0,Australia,5
1,Austria,2
2,Belgium,2
3,Canada,3
4,Denmark,2
5,Finland,3
6,France,12
7,Germany,13
8,Hong Kong,1
9,Ireland,2


### Alternative GROUP BY Syntax
Another thing to be aware of is that instead of specifying an actual column name to group by, we can group the data using the index of one of the columns already specified in the SELECT statement. These are 1-indexed (unlike Python, which is 0-indexed). So an alternative way to write the previous query would be:

In [4]:
q = ''' SELECT country, count(*) 
FROM customers
GROUP BY 1
'''
pd.read_sql(q,conn).head(10)

,country,count(*)
0,Australia,5
1,Austria,2
2,Belgium,2
3,Canada,3
4,Denmark,2
5,Finland,3
6,France,12
7,Germany,13
8,Hong Kong,1
9,Ireland,2


### Aliasing
An alias is a shorthand for a table or column name. Aliases reduce the amount of typing required to enter a query, and can result in both queries and results that are easier to read.

Aliases are especially useful with JOIN, GROUP BY, and aggregates (SUM, COUNT, etc.). For example, we could rewrite the previous query like this, so that the count of customers is called customer_count instead of COUNT(*):

In [5]:
q = '''
SELECT country, COUNT(*) as customer_count
FROM customers
GROUP BY country
'''
pd.read_sql(q, conn).head(10)

,country,customer_count
0,Australia,5
1,Austria,2
2,Belgium,2
3,Canada,3
4,Denmark,2
5,Finland,3
6,France,12
7,Germany,13
8,Hong Kong,1
9,Ireland,2


### Other notes on aliases:

- An alias only exists for the duration of the query.
- The keyword AS is optional in SQLite. So, you could just say COUNT(*) customer_count with the same outcome. Historically some forms of SQL required AS and others would not work with AS, but most work either way now. In a professional setting you will likely have a style guide indicating whether or not to use it.
### Other Aggregations
Aside from COUNT() some other useful aggregations include:

- MIN()
- MAX()
- SUM()
- AVG()
These are mainly useful when working with numeric data.

Payment Summary Statistics
In the cell below, we calculate various summary statistics about payments, grouped by customer.

In [57]:
Q=''' 
SELECT customerNumber,
    COUNT(*) AS number_payments,
    MIN(CAST(amount AS INT)) AS min_purchase,
    MAX(CAST(amount AS INT))  AS max_purchase,
    AVG(CAST(amount AS INT)) AS avg_purchase,
    SUM(CAST(amount AS INT)) AS total_spent
FROM payments
GROUP BY customerNumber
'''
pd.read_sql(q,conn)

,customerNumber,number_payments,min_purchase,max_purchase,avg_purchase,total_spent
0,103,3,14571.44,6066.78,7438.120000,22314.36
1,112,3,14191.12,33347.88,26726.993333,80180.98
2,114,4,44894.74,82261.22,45146.267500,180585.07
3,119,3,19501.82,49523.67,38983.226667,116949.68
4,121,4,1491.38,50218.95,26056.197500,104224.79
...,...,...,...,...,...,...
93,486,3,25833.14,5899.38,25908.863333,77726.59
94,487,2,12573.28,29997.09,21285.185000,42570.37
95,489,2,22275.73,7310.42,14793.075000,29586.15
96,495,2,59265.14,6276.60,32770.870000,65541.74


### Filtered Payment Summary Statistics with WHERE
Similar to before we used GROUP BY and aggregations, we can use WHERE to filter the data. For example, if we only wanted to include payments made in 2004:

In [58]:
q=''' 
SELECT customerNumber,
    COUNT(*) AS number_payments,
    MIN(CAST(amount AS INT)) AS min_purchase,
    MAX(CAST(amount AS INT))  AS max_purchase,
    AVG(CAST(amount AS INT)) AS avg_purchase,
    SUM(CAST(amount AS INT)) AS total_spent
FROM payments
WHERE substr(paymentDate,1,4) ='2004'
GROUP BY customerNumber
'''
pd.read_sql(q,conn)

,customerNumber,number_payments,min_purchase,max_purchase,avg_purchase,total_spent
0,103,2,1676,6066,3871.0,7742
1,112,2,14191,33347,23769.0,47538
2,114,2,44894,82261,63577.5,127155
3,119,2,19501,47924,33712.5,67425
4,121,2,17876,34638,26257.0,52514
...,...,...,...,...,...,...
83,486,2,5899,45994,25946.5,51893
84,487,1,12573,12573,12573.0,12573
85,489,1,7310,7310,7310.0,7310
86,495,1,6276,6276,6276.0,6276


In [59]:
#Teachers Code
q=''' 
SELECT customerNumber,
    COUNT(*) AS number_payments,
    MIN(CAST(amount AS INT)) AS min_purchase,
    MAX(CAST(amount AS INT))  AS max_purchase,
    AVG(CAST(amount AS INT)) AS avg_purchase,
    SUM(CAST(amount AS INT)) AS total_spent
FROM payments
WHERE strftime('%Y',paymentDate)='2004'
GROUP BY customerNumber
'''
pd.read_sql(q,conn)

,customerNumber,number_payments,min_purchase,max_purchase,avg_purchase,total_spent
0,103,2,1676,6066,3871.0,7742
1,112,2,14191,33347,23769.0,47538
2,114,2,44894,82261,63577.5,127155
3,119,2,19501,47924,33712.5,67425
4,121,2,17876,34638,26257.0,52514
...,...,...,...,...,...,...
83,486,2,5899,45994,25946.5,51893
84,487,1,12573,12573,12573.0,12573
85,489,1,7310,7310,7310.0,7310
86,495,1,6276,6276,6276.0,6276


### Some additional notes:

- Look at the difference in the first row values. It appears that customer 103 made 3 payments in the database overall, but only made 2 payments in 2004. So this row still represents the same customer as in the previous query, but it contains different aggregated information about that customer.
- This returned 88 rows rather than 98, because some of the customers are present in the overall database but did not make any purchases in 2004.
- Recall that you can filter based on something in a WHERE clause even if you do not SELECT that column. We are not displaying the paymentDate values because this would not make much sense in aggregate, but we can still use that column for filtering.

### The HAVING Clause
We can filter our aggregate views with the `HAVING` clause.The Having clause works similarly to the WHERE clause, except it is used to filter data selections on conditions after the GROUP BY STATEMENT
For example, if we wanted to filter to only select aggregated payment information about customers with average payment amounts over 50,000:


In [69]:
q=''' 
SELECT customerNumber,
    COUNT(*) AS number_payments,
    MIN(CAST(amount AS INT)) AS min_purchase,
    MAX(CAST(amount AS INT))  AS max_purchase,
    AVG(CAST(amount AS INT)) AS avg_purchase,
    SUM(CAST(amount AS INT)) AS total_spent
FROM payments
GROUP BY customerNumber
HAVING avg_purchase> 50000
'''
pd.read_sql(q,conn)

,customerNumber,number_payments,min_purchase,max_purchase,avg_purchase,total_spent
0,124,9,11044,111654,64909.333333,584184
1,141,13,20009,120166,55056.384615,715733
2,239,1,80375,80375,80375.000000,80375
3,298,2,47375,61402,54388.500000,108777
4,321,2,46781,85559,66170.000000,132340
5,450,1,59551,59551,59551.000000,59551


Note that in most flavors of SQL we can't use an alias in the HAVING clause. This is due to the internal order of execution of the SQL commands. So in most cases outside of SQLite you would need to write that query like this, repeating the aggregation code in the HAVING clause:

In [67]:
q=''' 
SELECT customerNumber,
    COUNT(*) AS number_payments,
    MIN(CAST(amount AS INT)) AS min_purchase,
    MAX(CAST(amount AS INT))  AS max_purchase,
    AVG(CAST(amount AS INT)) AS avg_purchase,
    SUM(CAST(amount AS INT)) AS total_spent
FROM payments
GROUP BY customerNumber
HAVING avg(amount) > 50000
'''
pd.read_sql(q,conn)

,customerNumber,number_payments,min_purchase,max_purchase,avg_purchase,total_spent
0,124,9,11044,111654,64909.333333,584184
1,141,13,20009,120166,55056.384615,715733
2,239,1,80375,80375,80375.000000,80375
3,298,2,47375,61402,54388.500000,108777
4,321,2,46781,85559,66170.000000,132340
5,450,1,59551,59551,59551.000000,59551


### Combining the WHERE and HAVING Clauses
We can also use the WHERE and HAVING clauses in conjunction with each other for more complex rules.

For example, let's say we want to filter based on customers who have made at least 2 purchases of over 50000 each.

In [74]:
q=''' 
SELECT customerNumber,
    COUNT(*) AS number_payments,
    MIN(CAST(amount AS INT)) AS min_purchase,
    MAX(CAST(amount AS INT))  AS max_purchase,
    AVG(CAST(amount AS INT)) AS avg_purchase,
    SUM(CAST(amount AS INT)) AS total_spent
FROM payments
WHERE CAST(amount AS INT) >50000
GROUP BY customerNumber
HAVING number_payments >=2
'''
pd.read_sql(q,conn)

,customerNumber,number_payments,min_purchase,max_purchase,avg_purchase,total_spent
0,124,5,55639,111654,87509.0,437545
1,141,5,59830,120166,85023.6,425118
2,151,2,58793,58841,58817.0,117634
3,363,2,50799,55425,53112.0,106224


We can also use the ORDER BY and LIMIT clauses in queries containing these complex rules. Say we want to find the customer with the lowest total amount spent, who nevertheless fits the criteria described above. That would be:

In [76]:
q=''' 
SELECT customerNumber,
    COUNT(*) AS number_payments,
    MIN(CAST(amount AS INT)) AS min_purchase,
    MAX(CAST(amount AS INT))  AS max_purchase,
    AVG(CAST(amount AS INT)) AS avg_purchase,
    SUM(CAST(amount AS INT)) AS total_spent
FROM payments
WHERE CAST(amount AS INT) >50000
GROUP BY customerNumber
HAVING number_payments >=2
ORDER BY total_spent
LIMIT 1
'''
pd.read_sql(q,conn)

,customerNumber,number_payments,min_purchase,max_purchase,avg_purchase,total_spent
0,363,2,50799,55425,53112.0,106224


### Summary
In this lesson, you learned how to use aggregate functions, aliases, and the HAVING clause to filter selections.